In [1]:
library(scater)
library(scDblFinder)
library(BiocParallel)
source("../tools/formating/formating.R")

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

In [2]:
# path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"
path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"

In [3]:
srat <- LoadSeurat(path)
srat

[1] "Inside LoadSeurat"


Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding variable feature information for RNA

Adding miscellaneous information for RNA

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction tsne

Adding cell embeddings for tsne

Adding miscellaneous information for tsne

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding graph RNA_nn

Adding graph RNA_snn

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



An object of class Seurat 
23341 features across 2500 samples within 1 assay 
Active assay: RNA (23341 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 3 dimensional reductions calculated: pca, tsne, umap

In [4]:
adata <- SeuratToAnndata(srat)
adata

AnnData object with n_obs × n_vars = 2500 × 23341
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id', 'percent.mt', 'percent.rb', 'percent.hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'scale.data'

In [5]:
adata$X

  [[ suppressing 32 column names ‘Xkr4’, ‘Rp1’, ‘Sox17’ ... ]]



2500 x 23341 sparse Matrix of class "dgCMatrix"
                                                                               
10X_P4_3_AAAGTAGAGATGCCAG . .  . 1 .  . . . . . 1 . . . . . . . . . . . 1 . 1 .
10X_P4_3_AACCGCGTCCAACCAA . .  . . 2  3 . . . . 2 . . . . 2 . . 1 . . . . . 1 .
10X_P4_3_AACTCCCGTCGGGTCT . .  . . 1  1 . 2 . . 1 . . . . . . . . . . . . . 1 .
10X_P4_3_AACTCTTAGTTGCAGG . .  . . 2  . . 1 . . . . . . . . . . . . . . . . . .
10X_P4_3_AACTCTTTCATAACCG . .  . . .  1 . 1 . . . . . 1 . 1 . . . . . . . . . .
10X_P4_3_AAGACCTAGATCCGAG . .  . 2 3  1 . . . . . . . . . . . . . . . . . . . .
10X_P4_3_AAGACCTAGGTGGGTT . .  . . 1  1 . 1 . . 1 . . . . 2 . . . . . . 1 . 3 .
10X_P4_3_AAGGAGCGTGCAACTT . .  . . .  . . . . . . . . . . . . . . . . . . . . .
10X_P4_3_AAGTCTGAGATAGTCA . .  . . .  1 . 1 . . . . . 1 . 1 . . . . 1 . . . 1 .
10X_P4_3_ACAGCTATCATACGGT . .  . . 1  . . . . . 1 . . 2 . . . . . . . . 1 . . .
10X_P4_3_ACATACGCAGCTCCGA . .  . . .  . . . . . . . . . . . . . . . . . 

In [11]:
obj <- srat
default_assay <- 'RNA'
if(paste0("nCount_", default_assay) %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) ==paste0("nCount_", default_assay)] <-"n_counts"
if(paste0("nFeature_", default_assay) %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) ==paste0("nFeature_", default_assay)] <-"n_genes"
if("percent.mt" %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) =="percent.mt"] <-"percent_mito"
obj@meta.data

,orig.ident,n_counts,n_genes,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,⋯,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id,percent_mito,percent.rb,percent.hb,percent.plat,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291,2853,10X_P4_3,Bladder,,M,3-M-8,0,0.21960937,⋯,NA,NA,0,CL:1001319,0,0,0,0,6,6
10X_P4_3_AACCGCGTCCAACCAA,10X,17520,4194,10X_P4_3,Bladder,,M,3-M-8,0,0.17671233,⋯,NA,NA,2,CL:1001319,0,0,0,0,5,5
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944,3347,10X_P4_3,Bladder,,M,3-M-8,0,0.19244438,⋯,NA,NA,5,CL:1001319,0,0,0,0,5,5
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594,3851,10X_P4_3,Bladder,,M,3-M-8,0,0.35638461,⋯,NA,NA,3,CL:1001428,0,0,0,0,3,3
10X_P4_3_AACTCTTTCATAACCG,10X,14594,3809,10X_P4_3,Bladder,,M,3-M-8,0,0.15711936,⋯,NA,NA,2,CL:1001319,0,0,0,0,5,5
10X_P4_3_AAGACCTAGATCCGAG,10X,19190,3608,10X_P4_3,Bladder,,M,3-M-8,0,0.23778009,⋯,NA,NA,6,CL:0000738,0,0,0,0,8,8
10X_P4_3_AAGACCTAGGTGGGTT,10X,24484,4347,10X_P4_3,Bladder,,M,3-M-8,0,0.32809182,⋯,NA,NA,1,CL:1001428,0,0,0,0,3,3
10X_P4_3_AAGGAGCGTGCAACTT,10X,5005,1960,10X_P4_3,Bladder,,M,3-M-8,0,0.19020979,⋯,NA,NA,0,CL:1001319,0,0,0,0,2,2
10X_P4_3_AAGTCTGAGATAGTCA,10X,6253,2236,10X_P4_3,Bladder,,M,3-M-8,0,0.22597153,⋯,NA,NA,4,CL:1001428,0,0,0,0,4,4


In [6]:
adata$layers['scale.data']

,Xkr4,Rp1,Sox17,Mrpl15,Lypla1,Tcea1,Rgs20,Atp6v1h,Oprk1,Npbwr1,⋯,MGC107098,LOC100039810,LOC100041256,LOC100039574,LOC100040911,Erdr1,Gfp-transgene,Cre-transgene,Tdtom-transgene,zsGreen-transgene
10X_P4_3_AAAGTAGAGATGCCAG,-0.03985626,-0.02,-0.1100388,1.4830777,-0.6858682,-0.99397070,-0.02616663,-0.5611504,-0.06324162,0,⋯,0,0,0,0,0,-0.87734445,0,0,0,0
10X_P4_3_AACCGCGTCCAACCAA,-0.03985626,-0.02,-0.1100388,-0.5859443,1.5248797,1.19737725,-0.02616663,-0.5611504,-0.06324162,0,⋯,0,0,0,0,0,0.02385663,0,0,0,0
10X_P4_3_AACTCCCGTCGGGTCT,-0.03985626,-0.02,-0.1100388,-0.5859443,0.9758965,0.26316585,-0.02616663,2.4407113,-0.06324162,0,⋯,0,0,0,0,0,-0.87734445,0,0,0,0
10X_P4_3_AACTCTTAGTTGCAGG,-0.03985626,-0.02,-0.1100388,-0.5859443,1.3562809,-0.99397070,-0.02616663,0.7638797,-0.06324162,0,⋯,0,0,0,0,0,-0.05441834,0,0,0,0
10X_P4_3_AACTCTTTCATAACCG,-0.03985626,-0.02,-0.1100388,-0.5859443,-0.6858682,0.15218917,-0.02616663,1.1158852,-0.06324162,0,⋯,0,0,0,0,0,-0.87734445,0,0,0,0
10X_P4_3_AAGACCTAGATCCGAG,-0.03985626,-0.02,-0.1100388,1.5901225,2.0467576,-0.07281902,-0.02616663,-0.5611504,-0.06324162,0,⋯,0,0,0,0,0,-0.87734445,0,0,0,0
10X_P4_3_AAGACCTAGGTGGGTT,-0.03985626,-0.02,-0.1100388,-0.5859443,0.3083490,-0.24183850,-0.02616663,0.5393526,-0.06324162,0,⋯,0,0,0,0,0,0.31431138,0,0,0,0
10X_P4_3_AAGGAGCGTGCAACTT,-0.03985626,-0.02,-0.1100388,-0.5859443,-0.6858682,-0.99397070,-0.02616663,-0.5611504,-0.06324162,0,⋯,0,0,0,0,0,2.33302580,0,0,0,0
10X_P4_3_AAGTCTGAGATAGTCA,-0.03985626,-0.02,-0.1100388,-0.5859443,-0.6858682,1.10383954,-0.02616663,2.5083188,-0.06324162,0,⋯,0,0,0,0,0,2.63003163,0,0,0,0
10X_P4_3_ACAGCTATCATACGGT,-0.03985626,-0.02,-0.1100388,-0.5859443,0.7298851,-0.99397070,-0.02616663,-0.5611504,-0.06324162,0,⋯,0,0,0,0,0,0.74729636,0,0,0,0


In [ ]:
srat[['RNA']]@counts

In [7]:
mat <- Seurat::GetAssayData(object=srat, assay='RNA', layer='X')

ERROR: Error in match.arg(arg = layer): 'arg' should be one of “data”, “scale.data”, “counts”


In [ ]:
default_assay <- DefaultAssay(srat)
default_assay

In [ ]:
adata <- LoadAnndata("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad")
adata

In [ ]:
adata$obsm['X_pca']

In [ ]:
Embeddings(object = srat, reduction = "pca")

In [ ]:
max(srat[['RNA']]@counts)

In [ ]:
min(srat[['RNA']]@counts)

In [ ]:
doublet_annnotation <- AnnotateDroplet(srat[[default_assay]]@counts)

In [ ]:
doublet_annnotation$doublet_score

In [ ]:
srat[["Doublet_score"]] <- doublet_annnotation$doublet_score
srat[["Doublet_class"]] <- doublet_annnotation$doublet_class

In [ ]:
data_mat = srat[['RNA']]@counts

In [ ]:
# -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
srat[["Doublet_score"]] <- doublet_score

In [ ]:
srat[["Doublet_class"]] <- doublet_class

In [ ]:
srat[[]]

In [ ]:
dim(srat[['RNA']]@counts)
dim(srat)
dim(srat@meta.data)

In [ ]:
if(!paste0("nCount_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nCount_", default_assay)]] <- colSums(x = srat[[default_assay]], slot = "counts")  # nCount of the default assay
if(!paste0("nFeature_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nFeature_", default_assay)]] <- colSums(x = GetAssayData(object = srat[[default_assay]], slot = "counts") > 0)  # nFeature of the default assay

In [ ]:
head(srat@meta.data)

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)

In [ ]:
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat1@meta.data

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', dest = "h5ad" , overwrite = TRUE)
adata_path

In [ ]:
# Calculate the percentage of mitocondrial per cell and add to the metadata.
if(! "percent.mt" %in% names(x = srat[[]])) srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-")
# Calculate the proportion gene expression that comes from ribosomal proteins.
if(! "percent.rb" %in% names(x = srat[[]])) srat[["percent.rb"]] <- PercentageFeatureSet(srat, pattern = "^RP[SL]")

# Percentage hemoglobin genes - includes all genes starting with HB except HBP.
if(! "percent.hb" %in% names(x = srat[[]])) srat[["percent.hb"]] <- PercentageFeatureSet(srat, pattern = "^HB[^(P)]")
if(! "percent.plat" %in% names(x = srat[[]])) srat[["percent.plat"]] <- PercentageFeatureSet(srat, pattern = "PECAM1|PF4")
head(srat@meta.data)

In [ ]:
# Add the doublet annotation
doublets <- read.table(here::here('api/tools/qc/scrublet_calls.tsv'), header = F, row.names = 1)
colnames(doublets) <- c("Doublet_score", "Is_doublet")
srat <- AddMetaData(srat, doublets)
# srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
head(srat@meta.data)

In [14]:
srat@meta.data

,orig.ident,n_counts,nFeature_RNA,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,⋯,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id,percent.mt,percent.rb,percent.hb,percent.plat,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291,2853,10X_P4_3,Bladder,,M,3-M-8,0,0.21960937,⋯,NA,NA,0,CL:1001319,0,0,0,0,6,6
10X_P4_3_AACCGCGTCCAACCAA,10X,17520,4194,10X_P4_3,Bladder,,M,3-M-8,0,0.17671233,⋯,NA,NA,2,CL:1001319,0,0,0,0,5,5
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944,3347,10X_P4_3,Bladder,,M,3-M-8,0,0.19244438,⋯,NA,NA,5,CL:1001319,0,0,0,0,5,5
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594,3851,10X_P4_3,Bladder,,M,3-M-8,0,0.35638461,⋯,NA,NA,3,CL:1001428,0,0,0,0,3,3
10X_P4_3_AACTCTTTCATAACCG,10X,14594,3809,10X_P4_3,Bladder,,M,3-M-8,0,0.15711936,⋯,NA,NA,2,CL:1001319,0,0,0,0,5,5
10X_P4_3_AAGACCTAGATCCGAG,10X,19190,3608,10X_P4_3,Bladder,,M,3-M-8,0,0.23778009,⋯,NA,NA,6,CL:0000738,0,0,0,0,8,8
10X_P4_3_AAGACCTAGGTGGGTT,10X,24484,4347,10X_P4_3,Bladder,,M,3-M-8,0,0.32809182,⋯,NA,NA,1,CL:1001428,0,0,0,0,3,3
10X_P4_3_AAGGAGCGTGCAACTT,10X,5005,1960,10X_P4_3,Bladder,,M,3-M-8,0,0.19020979,⋯,NA,NA,0,CL:1001319,0,0,0,0,2,2
10X_P4_3_AAGTCTGAGATAGTCA,10X,6253,2236,10X_P4_3,Bladder,,M,3-M-8,0,0.22597153,⋯,NA,NA,4,CL:1001428,0,0,0,0,4,4


In [9]:
names(srat@meta.data)[names(srat@meta.data) =="nCount_RNA"] <-"n_counts"
srat@meta.data

,orig.ident,n_counts,nFeature_RNA,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,⋯,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id,percent.mt,percent.rb,percent.hb,percent.plat,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291,2853,10X_P4_3,Bladder,,M,3-M-8,0,0.21960937,⋯,NA,NA,0,CL:1001319,0,0,0,0,6,6
10X_P4_3_AACCGCGTCCAACCAA,10X,17520,4194,10X_P4_3,Bladder,,M,3-M-8,0,0.17671233,⋯,NA,NA,2,CL:1001319,0,0,0,0,5,5
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944,3347,10X_P4_3,Bladder,,M,3-M-8,0,0.19244438,⋯,NA,NA,5,CL:1001319,0,0,0,0,5,5
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594,3851,10X_P4_3,Bladder,,M,3-M-8,0,0.35638461,⋯,NA,NA,3,CL:1001428,0,0,0,0,3,3
10X_P4_3_AACTCTTTCATAACCG,10X,14594,3809,10X_P4_3,Bladder,,M,3-M-8,0,0.15711936,⋯,NA,NA,2,CL:1001319,0,0,0,0,5,5
10X_P4_3_AAGACCTAGATCCGAG,10X,19190,3608,10X_P4_3,Bladder,,M,3-M-8,0,0.23778009,⋯,NA,NA,6,CL:0000738,0,0,0,0,8,8
10X_P4_3_AAGACCTAGGTGGGTT,10X,24484,4347,10X_P4_3,Bladder,,M,3-M-8,0,0.32809182,⋯,NA,NA,1,CL:1001428,0,0,0,0,3,3
10X_P4_3_AAGGAGCGTGCAACTT,10X,5005,1960,10X_P4_3,Bladder,,M,3-M-8,0,0.19020979,⋯,NA,NA,0,CL:1001319,0,0,0,0,2,2
10X_P4_3_AAGTCTGAGATAGTCA,10X,6253,2236,10X_P4_3,Bladder,,M,3-M-8,0,0.22597153,⋯,NA,NA,4,CL:1001428,0,0,0,0,4,4


In [13]:
names(srat@meta.data)

[1] "orig.ident"                   "n_counts"                    
 [3] "nFeature_RNA"                 "channel"                     
 [5] "tissue"                       "subtissue"                   
 [7] "mouse.sex"                    "mouse.id"                    
 [9] "percent.ercc"                 "percent.ribo"                
[11] "free_annotation"              "cell_ontology_class"         
[13] "res.0.4"                      "previous_free_annotation"    
[15] "previous_cell_ontology_class" "cluster.ids"                 
[17] "cell_ontology_id"             "percent.mt"                  
[19] "percent.rb"                   "percent.hb"                  
[21] "percent.plat"                 "RNA_snn_res.0.5"             
[23] "seurat_clusters"

In [ ]:
nFeature_min=200
nFeature_max=0
percent_mt_max=5
percent_rb_min=0
srat <- subset(srat, subset = paste0("nFeature_", default_assay) > nFeature_min & percent.mt < percent_mt_max)
if(nFeature_max != 0) srat <- subset(srat, subset = paste0("nFeature_", default_assay) < nFeature_max)
if(percent_rb_min != 0)  srat <- subset(srat, subset = percent.rb > percent_rb_min)

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True' | is.na(Is_doublet))

In [ ]:
#' Regularise dataframe
#'
#' This function checks if certain columns of a dataframe is of a single value
#' and drop them if required
#'
#' @param df Input data frame, usually cell metadata table (data.frame-like
#'   object)
#' @param drop_single_values Drop columns with only a single value (logical)
#'
#' @return Dataframe
.regularise_df <- function(df, drop_single_values=FALSE, drop_na_values=TRUE) {
  if (ncol(df) == 0) df[["name"]] <- rownames(df)
  if (drop_single_values) {
    k_singular <- sapply(df, function(x) length(unique(x)) == 1)
    if (sum(k_singular) > 0) {
      warning(
        paste("Dropping single category variables:"),
        paste(colnames(df)[k_singular], collapse = ", ")
      )
    }
    df <- df[, !k_singular, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
 if (drop_na_values) {
    k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
    if (sum(k_na) > 0) {
      warning(
        paste("Dropping NA category variables:"),
        paste(colnames(df)[k_na], collapse = ", ")
      )
    }
    df <- df[, !k_na, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
  return(df)
}

In [ ]:
output_path_rds <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds'
saveRDS(srat, output_path_rds)
srat1 <- LoadSeurat(output_path_rds)
srat1

In [ ]:
k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
k_na
sum(k_na)

In [ ]:
srat_qc@meta.data %>% is.na()

In [ ]:
k_singular <- sapply(srat_qc@meta.data, function(x) length(unique(x)) == 1)
k_singular

In [ ]:
srat_qc@meta.data

In [ ]:
srat@meta.data <- .regularise_df(srat@meta.data, drop_single_values=FALSE, drop_na_values=TRUE)
srat@meta.data

In [ ]:
srat_qc@meta.data <- .regularise_df(srat_qc@meta.data, drop_single_values = FALSE)
srat_qc@meta.data

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_3 <- LoadH5Seurat(output_path)
srat_3

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat1, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_2 <- LoadH5Seurat(output_path)
srat_2

In [ ]:
srat1[["meta.data"]]

In [ ]:
srat1@meta.data

In [ ]:
devtools::install_github("cellgeni/sceasy")

In [ ]:
sceasy::convertFormat(srat1, from="seurat", to="anndata",
                       outFile='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')

In [ ]:
adata <- LoadAnndata('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')
adata

In [ ]:
adata$obs

In [ ]:
adata$X

In [ ]:
srat_qc <- CreateSeuratObject(counts=srat[['RNA']]@counts, meta.data=srat@meta.data, project = Project(srat))
srat_qc

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_qc_1 <- LoadH5Seurat(output_path)
srat_qc_1

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')

In [ ]:
srat[["meta.data"]]

In [ ]:
Cells(srat)

In [ ]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))
srat
dim(srat)
dim(srat@meta.data)

In [ ]:
srat[['Doublet_score']][sapply(srat[['Doublet_score']], is.na)] <- 0
srat[['Doublet_score']]

In [ ]:
srat[['Doublet_score']] <- !is.na(srat[['Doublet_score']])


In [ ]:
# srat <- subset(srat, subset = percent.rb >= 0)
srat <- subset(srat, subset = is.na(Is_doublet))
srat

In [ ]:
srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')
srat

In [ ]:
srat@meta.data

In [ ]:
'nCount_RNA' %in% names(srat@meta.data)

In [ ]:
result <- GetMetadataFromSeurat(path)
result

In [ ]:
suffix <- tolower(get_suffix(path))
suffix

In [ ]:
seurat_object <- LoadSeurat(path)
seurat_object

In [ ]:
'umap' %in% names(seurat_object@reductions)

In [ ]:
genes <- rownames(seurat_object)
typeof(genes)
genes

In [ ]:
cells <- Cells(seurat_object)
typeof(cells)
cells

In [ ]:
pca <- Embeddings(object = seurat_object, reduction = "pca")
typeof(pca)
pca

In [ ]:
name <- load(path)
seurat_object <- UpdateSeuratObject(get(name))

In [ ]:
seurat_object

In [ ]:
features <- seurat_object[['RNA']]@var.features
typeof(features)
features

In [ ]:
#获取当前用的Idents
Idents(seurat_object)
levels(seurat_object)

In [ ]:
#总数目
nCells <- ncol(seurat_object)
nGenes <- nrow(seurat_object)

nCells
nGenes

In [ ]:
dim=1:10
class(dim)

In [ ]:
x <- 1:10
x

In [ ]:
class(x)

In [ ]:
devtools::install_github('JiekaiLab/dior')